In [153]:
import pickle
import seaborn as sns
sns.set()
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import glob
from scipy.signal import savgol_filter
import imageio
import pickle
from moviepy.editor import *
import cv2

my_dpi = 96

In [163]:
policy_mode = "hzdrl"
adv_mode = "forward"
base_path = "log/2000/adv/"+policy_mode+"_"+adv_mode+"/fix_1.0_/"

In [164]:
filename = glob.glob(base_path+"*.mp4")[0]
vid = imageio.get_reader(filename, 'ffmpeg')
images = []
for i, im in enumerate(vid):
    images += [im]
images = np.array(images)

In [165]:
print images.shape

(5000, 500, 500, 3)


In [166]:
class Prep_Data():
    def __init__(self, path=None, filter_size=401):
        with open(path, "rb") as p:
            data = pickle.load(p)
        
        self.vels = data["vels"]
        self.kds = data["kds"]
        self.dvs = data["dvs"]        
        self.states = data["states"]
        self.vels_smooth = savgol_filter(self.vels, filter_size, 3)
        self.label = path.split("data/")[-1].split("_/")[0]
        self.advs = data["adversaries"]

In [167]:
data = Prep_Data(path=glob.glob(base_path+"*.pkl")[0])

In [168]:
def make_vel_plot(vels):
    sns.set_style("darkgrid")
    sns.set_context("paper", font_scale=2)
    blues = sns.dark_palette("blue", 8)
    
    f, ax = plt.subplots(figsize=(10,5))
    times = np.arange(vels.shape[0])*0.002
    ax.plot(times, vels, "-", color=blues[7], linewidth=1, label="hip velocity")
    if vels.shape[0] > 2:
        ax.plot(times[vels.shape[0]-1], vels[-1], "o", color=blues[7], linewidth=6, label="current hip velocity")
    else:
        ax.plot(0., 0., "o", color=blues[7], linewidth=6, label="current hip velocity")
    ax.set_ylim(0., 1.5)
    ax.set_xlim(0, 11)
    ax.legend(loc=4)
    
    f.text(0.5, 0.05, 'time(s)', ha='center')
    f.text(0.05, 0.5, 'hip velocity (m/s)', va='center', rotation='vertical')
    
    return f

In [169]:
video_path = base_path + "frames/"
if not os.path.exists(video_path):
    os.makedirs(video_path)
for i in range(5000):
    if i % 25 == 0:
        fig = make_vel_plot(data.vels_smooth[0:i])
        plt.savefig(video_path+str("%05d" % i)+".jpg")
        plt.clf()
        if i > 1000 and i % 1000 < 500: #data.advs[i-1000][0] != 0.:
            if adv_mode == "forward":
                image = cv2.arrowedLine(images[i], (150, 200), (250, 200), (255,69,0), 5)
            else:
                image = cv2.arrowedLine(images[i], (400, 200), (300, 200), (255,69,0), 5)
        else:
            image = images[i]
        img = imageio.imread(video_path+str("%05d" % i)+".jpg")
        img = cv2.resize(img, (500, 250))
        frame = np.concatenate([image, img], axis=0)
        imageio.imwrite(video_path+str("%05d" % i)+".jpg", frame)
    

In [170]:
video_path = base_path + "frames/"
if not os.path.exists(video_path):
    os.makedirs(video_path)
frames = []
for i in range(5000):
    if i % 25 == 0:
        frames += [imageio.imread(video_path+str("%05d" % i)+".jpg")]

In [171]:
clip = ImageSequenceClip(frames, fps=20)
clip.write_videofile(base_path+policy_mode+"_"+adv_mode+"_demo.mp4", audio=False)

[MoviePy] >>>> Building video log/2000/adv/hzdrl_forward/fix_1.0_/hzdrl_forward_demo.mp4
[MoviePy] Writing video log/2000/adv/hzdrl_forward/fix_1.0_/hzdrl_forward_demo.mp4


100%|██████████| 201/201 [00:00<00:00, 320.67it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: log/2000/adv/hzdrl_forward/fix_1.0_/hzdrl_forward_demo.mp4 



In [22]:
filename = "log/1000/adv/hzd/fix_0.8_/openaigym.video.0.29864.video000000.mp4"
vid = imageio.get_reader(filename, 'ffmpeg')
images = []
for i, im in enumerate(vid):
    images += [im]
images = np.array(images)

In [23]:
pklfile = "log/1000/adv/hzd/fix_0.8_/fix_0.8.pkl"
with open(pklfile, "rb") as p:
    data = pickle.load(p)
    adversaries = data["adversaries"]
print images.shape, adversaries.shape

(3000, 500, 500, 3) (3000, 2)


In [28]:
frames = []
for i in range(images.shape[0]):
    frame = images[i]
    if i > 1000 and np.any(adversaries[i-300] > 0):
        frame = cv2.arrowedLine(frame, (100, 200), (250, 200), (255,69,0), 5)
    frames += [frame]
clip1 = ImageSequenceClip(frames[0:1000], fps=200)
clip2 = ImageSequenceClip(frames[1000:1600], fps=50)
finalclip = concatenate_videoclips([clip1,clip2])
finalclip.write_videofile("demo_hzd.mp4", audio=False)

[MoviePy] >>>> Building video demo_hzd.mp4
[MoviePy] Writing video demo_hzd.mp4


100%|██████████| 3400/3400 [00:06<00:00, 545.06it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: demo_hzd.mp4 



In [31]:
filename = "log/1000/adv/hzdrl/fix_1.3_/openaigym.video.2.29278.video000000.mp4"
vid = imageio.get_reader(filename, 'ffmpeg')
images = []
for i, im in enumerate(vid):
    images += [im]
images = np.array(images)

In [32]:
pklfile = "log/1000/adv/hzdrl/fix_1.3_/fix_1.3.pkl"
with open(pklfile, "rb") as p:
    data = pickle.load(p)
    adversaries = data["adversaries"]
print images.shape, adversaries.shape

(3000, 500, 500, 3) (3000, 2)


In [34]:
frames = []
for i in range(images.shape[0]):
    frame = images[i]
    if i > 1000 and np.any(adversaries[i] > 0):
        frame = cv2.arrowedLine(frame, (100, 200), (250, 200), (255,69,0), 5)
    frames += [frame]
clip1 = ImageSequenceClip(frames[0:1000], fps=200)
clip2 = ImageSequenceClip(frames[1000:1500], fps=50)
clip3 = ImageSequenceClip(frames[1500:2000], fps=200)
clip4 = ImageSequenceClip(frames[2000:2500], fps=50)
clip5 = ImageSequenceClip(frames[2500:], fps=200)
finalclip = concatenate_videoclips([clip1,clip2,clip3,clip4,clip5])
finalclip.write_videofile("demo_hzdrl.mp4", audio=False)

[MoviePy] >>>> Building video demo_hzdrl.mp4
[MoviePy] Writing video demo_hzdrl.mp4


100%|██████████| 6000/6000 [00:10<00:00, 558.09it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: demo_hzdrl.mp4 



(10000, 500, 500, 3)
